In [1]:
# Contains source code from PyGad, licence shown below:

# "Copyright GeneticAlgorithmPython Contributors
# BSD 3-Clause "New" or "Revised" License
#Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

#    1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#    2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
#    3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE."

In [2]:
# This also uses DoWhy, licence shown below.

# MIT License

#    Copyright (c) PyWhy contributors. All rights reserved.

#   Permission is hereby granted, free of charge, to any person obtaining a copy
#    of this software and associated documentation files (the "Software"), to deal
#    in the Software without restriction, including without limitation the rights
#    to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#    copies of the Software, and to permit persons to whom the Software is
#    furnished to do so, subject to the following conditions:

#    The above copyright notice and this permission notice shall be included in all
#    copies or substantial portions of the Software.

#    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#    IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#    FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#    AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#    LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#    OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#    SOFTWARE

In [3]:
!pip install tensorflow
!pip install numpy
!pip install dowhy
!pip install tensorflow_probability
!pip install pygad
!pip install pandas

In [4]:
# Prototype Genetic causal learning algorithm.

# based on https://www.pywhy.org/dowhy/v0.9.1/example_notebooks/dowhy-conditional-treatment-effects.html

# Note that this needs updating to use GCN rather than the current Conditional Average Treatment effects.

# References:
# https://www.pywhy.org/dowhy/v0.9.1/user_guide/gcm_based_inference/introduction.html

# https://www.pywhy.org/dowhy/v0.9.1/example_notebooks/dowhy-conditional-treatment-effects.html

# https://pygad.readthedocs.io/en/latest/README_pygad_kerasga_ReadTheDocs.html#functions-in-the-pygad-kerasga-module
# Note that this is going to be updated as soon as possible, to use a conditional loop within the genetic algorithm,
# so that it can take into account p-values and confidence intervals during the genetic training process.
# The basic idea is that while it will be an 'evolving algorithm', it will also be one that can and will
# explain what it's decision is, why and how it came to decide that.
# It might also be possible to implement a conditional loop, so that it can redesign it's own graphical models,
# and then try again, but that is a potentially longer term objective.
# Author: T.N.Toseland.

import tensorflow.keras
import pygad.kerasga
import numpy
import dowhy
import networkx as nx
import pandas as pd

import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets

import econml
import warnings
import networkx as nx, numpy as np, pandas as pd

import dowhy.gcm as gcm
warnings.filterwarnings('ignore')

BETA = 10

data = dowhy.datasets.linear_dataset(BETA, num_common_causes=4, num_samples=10000,
                                    num_instruments=2, num_effect_modifiers=2,
                                     num_treatments=1,
                                    treatment_is_binary=False,
                                    num_discrete_common_causes=2,
                                    num_discrete_effect_modifiers=0,
                                    one_hot_encode=False)
df=data['df']
print(df.head())
print("True causal estimate is", data["ate"])

df1 = pd.DataFrame.to_numpy(df)
df2 = pd.DataFrame.to_numpy(df)


2023-06-06 23:28:02.073936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 23:28:02.278753: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 23:28:03.019557: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-06 23:28:03.019596: W tensorflow/compiler/xla/stream_executor/platform/default/dso_lo

         X0        X1   Z0        Z1        W0        W1 W2 W3         v0  \
0 -2.554940 -0.167255  1.0  0.500447  1.423311  1.513411  3  2  37.855728   
1 -1.887606 -0.318242  1.0  0.449446  1.254425 -0.233838  3  3  38.396949   
2 -0.940927 -0.475711  1.0  0.253097  0.555077 -0.540435  0  2  23.787454   
3  0.604764  0.312491  1.0  0.178406 -0.449954 -1.255425  0  2  22.151487   
4 -1.547137 -1.452978  0.0  0.566060 -1.994247  0.401066  1  1  14.933177   

            y  
0   62.640323  
1  118.050478  
2  126.145642  
3  292.293391  
4   -7.192869  
True causal estimate is 3.934879097279504


In [5]:
# This is a prototype GCM model in place of the previous CATE one, i.e. Graphical Causal Model.

# It is not yet completed, but this is going to form the foundations of the causal learning algorithm,
# in which the genetic algorithm will feed into, and the genetic algorithm is used for estimation purposes.

import dowhy.gcm
import networkx as nx
from dowhy.gcm.uncertainty import estimate_variance
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets

import econml
import warnings
warnings.filterwarnings('ignore')




In [6]:
causal_model = CausalModel(data=data["df"],
                    treatment=data["treatment_name"], outcome=data["outcome_name"],
                    graph=data["gml_graph"])

In [ ]:
causal_model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = causal_model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
estimate = causal_model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print("Causal Estimate is " + str(estimate.value))

In [ ]:
import tensorflow.keras
import pygad.kerasga
import numpy
import pygad
import tensorflow as tf
import tensorflow_probability as tfp
import dowhy.gcm
# Custom fitness function modified to use a GCM approach to causal learning.

def fitness_func(ga_instance, solution, sol_idx):
    global data_inputs, data_outputs, keras_ga, model
    
    # Causal mechanisms used in the causal model, these can be changed depending on the data and on
    # the structure of the graph used.
    causal_graph = nx.DiGraph([('X0', 'X1'), ('Z1', 'y'),('Z1','Z0'),('Z1','y'),('y','W1'),('y','W3')])
    causal_model = gcm.ProbabilisticCausalModel(causal_graph,df)
    gcm.auto.assign_causal_mechanisms(causal_model, df)
    # Need to add causal mechanisms for W3, W1, and v0.
    causal_model.set_causal_mechanism('X0', gcm.stochastic_models.BayesianGaussianMixtureDistribution())
    causal_model.set_causal_mechanism('y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
    causal_model.set_causal_mechanism('X1', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
    causal_model.set_causal_mechanism('Z1', gcm.EmpiricalDistribution())
    causal_model.set_causal_mechanism('Z0', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
    causal_model.set_causal_mechanism('W1', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
    causal_model.set_causal_mechanism('W3', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
  
    estimate
    gcm.fit(causal_model, df)
        
    contributions1 = gcm.intrinsic_causal_influence(causal_model, 'Z0',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    contributions1
    
    contributions2 = gcm.intrinsic_causal_influence(causal_model, 'Z1',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    contributions2
    
    contributions3 = gcm.intrinsic_causal_influence(causal_model, 'X0',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    contributions3
    
    contributions4 = gcm.intrinsic_causal_influence(causal_model, 'y',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    contributions4
    
    contributions5 = gcm.intrinsic_causal_influence(causal_model, 'W1',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    
    contributions6 = gcm.intrinsic_causal_influence(causal_model, 'W3',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
    
    contributions7 = gcm.intrinsic_causal_influence(causal_model, 'X1',
                                              gcm.ml.create_linear_regressor(),
                                              lambda x, _: estimate_variance(x))
 
    predictions = pygad.kerasga.predict(model=model,
                                        solution=solution,
                                        data=data_inputs)

    mae = tensorflow.keras.losses.MeanAbsoluteError()
    abs_error = mae(data_outputs, predictions).numpy() + 0.00000001
    print("Results for Z0 confidence intervals",contributions1)
    print("Results for Z1 confidence interval",contributions2)
    print("Results for X0 confidence interval",contributions3)
    print("Results for y confidence interval",contributions4)
    print("Results for W1 confidence interval",contributions5)
    print("Results for W3 confidence interval",contributions6)
    print("Results for X1 confidence interval",contributions6)
    solution_fitness = 1.0/abs_error
    
    return solution_fitness

def callback_generation(ga_instance):
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))

input_layer  = tensorflow.keras.layers.Input(10)
dense_layer1 = tensorflow.keras.layers.Dense(5, activation="relu")(input_layer)
tfp.layers.DenseReparameterization(512, activation=tf.nn.relu),
tf.keras.layers.BatchNormalization()
tfp.layers.Convolution1DReparameterization(
        64, kernel_size=5, padding='SAME', activation=tf.nn.selu), # changed from relu.

output_layer = tensorflow.keras.layers.Dense(1, activation="selu")(dense_layer1)

model = tensorflow.keras.Model(inputs=input_layer, outputs=output_layer)

keras_ga = pygad.kerasga.KerasGA(model=model,
                                 num_solutions=60)

# Data inputs
data_inputs = df1

# Data outputs
data_outputs = df2

# Prepare the PyGAD parameters. Check the documentation for more information: https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html#pygad-ga-class
num_generations = 10 # Number of generations.
num_parents_mating = 6 # Number of solutions to be selected as parents in the mating pool.
initial_population = keras_ga.population_weights # Initial population of network weights

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation,parallel_processing=["thread", 32],save_solutions=True,
                      save_best_solutions=True)
# Modified the above to enable parallel processing, note that it might be possible to use a GPU for this.
ga_instance.summary(print_step_parameters=True,
                    print_parameters_summary=True)

ga_instance.run()

# After the generations complete, some plots are showed that summarize how the outputs/fitness values evolve over generations.
ga_instance.plot_fitness(title="Causal Learning PyGad & Keras - Iteration vs. Fitness", linewidth=4)

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

# Make prediction based on the best solution.
predictions = pygad.kerasga.predict(model=model,
                                    solution=solution,
                                    data=data_inputs)
print("Predictions : \n", predictions)

mae = tensorflow.keras.losses.MeanAbsoluteError()
abs_error = mae(data_outputs, predictions).numpy()
print("Absolute Error : ", abs_error)